In [39]:
!git clone https://github.com/nickxir12/MyCLIP_first_repo.git /kaggle/working/open-clip

fatal: destination path '/kaggle/working/open-clip' already exists and is not an empty directory.


In [40]:
# Step 1: Delete the old repo (if it exists)
!rm -rf /kaggle/working/open-clip  

# Step 2: Clone the latest version from GitHub
!git clone https://github.com/nickxir12/MyCLIP_first_repo.git /kaggle/working/open-clip

Cloning into '/kaggle/working/open-clip'...
remote: Enumerating objects: 3741, done.
remote: Counting objects: 100% (3741/3741), done.
remote: Compressing objects: 100% (1464/1464), done.
remote: Total 3741 (delta 2271), reused 3641 (delta 2175), pack-reused 0 (from 0)
Receiving objects: 100% (3741/3741), 16.76 MiB | 7.09 MiB/s, done.
Resolving deltas: 100% (2271/2271), done.


In [41]:
#!pip install open_clip_torch
!pip install braceexpand
!pip install webdataset

In [42]:
!pip install -r /kaggle/working/open-clip/requirements.txt

In [43]:
import os
import sys
sys.path.append("/kaggle/working/open-clip/src")
sys.path.append("/kaggle/working/open-clip/src/open_clip")
sys.path.append("/kaggle/working/open-clip/src/open_clip_train/my_metrics")

In [44]:
# import importlib
# import my_metrics
# importlib.reload(my_metrics)
# print(dir(my_metrics))  # Now should include 'batch', 'get_all_embeddings', etc.

In [45]:
from my_metrics import compute_consistency_score,evaluate_model

#Does below work?
from my_metrics import batch,get_all_embeddings,itm_eval,compute_consistency_score

In [46]:
import open_clip
import open_clip_train

In [47]:
import numpy as np
from PIL import Image
from tqdm import tqdm
import json

In [48]:
import torch
import os
import open_clip
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [49]:
#open_clip.list_pretrained()

In [50]:
#                Preparing train datasaet

In [51]:
import json
import pandas as pd

# Load Karpathy JSON file
karpathy_json_path = "/kaggle/input/karpathy-splits/dataset_flickr30k.json"
with open(karpathy_json_path, "r") as f:
    karpathy_data = json.load(f)

# Extract training set
train_data = [item for item in karpathy_data["images"] if item["split"] == "train"]

# Prepare data for CSV
train_records = []
for item in train_data:
    img_filename = f"/kaggle/input/flickr30k/Images/{item['filename']}"
    for sentence in item["sentences"]:
        caption = sentence["raw"]
        train_records.append({"image": img_filename, "caption": caption})

# Convert to DataFrame
df_train = pd.DataFrame(train_records)

# Save to CSV (formatted properly)
csv_path = "/kaggle/working/train_data_karpathy.csv"
df_train.to_csv(csv_path, index=True, index_label="id")

print(f"✅ CSV file saved at: {csv_path}")


✅ CSV file saved at: /kaggle/working/train_data_karpathy.csv


In [52]:
df_train.head()

,image,caption
0,/kaggle/input/flickr30k/Images/1000092795.jpg,Two young guys with shaggy hair look at their ...
1,/kaggle/input/flickr30k/Images/1000092795.jpg,"Two young, White males are outside near many b..."
2,/kaggle/input/flickr30k/Images/1000092795.jpg,Two men in green shirts are standing in a yard.
3,/kaggle/input/flickr30k/Images/1000092795.jpg,A man in a blue shirt standing in a garden.
4,/kaggle/input/flickr30k/Images/1000092795.jpg,Two friends enjoy time spent together.


In [53]:
karpathy_json_path = "/kaggle/input/karpathy-splits/dataset_flickr30k.json"
with open(karpathy_json_path, "r") as f:
    karpathy_data = json.load(f)

# Extract test set
test_data = [item for item in karpathy_data["images"] if item["split"] == "test"]

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [55]:
#Standard loading
# model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
# model = model.to(device)
# model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
# tokenizer = open_clip.get_tokenizer('ViT-B-32')

In [56]:
# # FOR CYCLIP PRE TRAINED THAT I HAVE DOWNLOADED LOCALLY
# model_clip_3M, _, preprocess = open_clip.create_model_and_transforms(
#     model_name="RN50",
#     pretrained=None,  # Don't load default weights
#     precision='fp32', # or 'amp' for mixed precision
#     device=device
# )

In [57]:
# # Load the checkpoint - OG CLIP - 3M
# ckpt = torch.load("/kaggle/input/clip-3m-from-cyclip/clip-3M.pt/best.pt", map_location=device)
# state_dict = ckpt["state_dict"]

# # Remove 'module.' prefix from keys
# new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# # Load into your model
# model_clip_3M.load_state_dict(new_state_dict)
# model_clip_3M.to(device)
# model_clip_3M.eval()

In [58]:
# FOR CYCLIP PRE TRAINED THAT I HAVE DOWNLOADED LOCALLY
# model_Cyclip_3M, _, preprocess = open_clip.create_model_and_transforms(
#     model_name="RN50",
#     pretrained=None,  # Don't load default weights
#     precision='fp32', # or 'amp' for mixed precision
#     device=device
# )

In [59]:
# # Load the checkpoint - CY CLIP - 3M
# ckpt = torch.load("/kaggle/input/cyclip-3m/CYCLIP-3M_best.pt", map_location=device)
# state_dict = ckpt["state_dict"]

# # Remove 'module.' prefix from keys
# new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# # Load into your model
# model_Cyclip_3M.load_state_dict(new_state_dict)
# model_Cyclip_3M.to(device)
# model_Cyclip_3M.eval()

In [60]:
#Tokenizer for my models
tokenizer = open_clip.get_tokenizer('RN50')

In [61]:
#FOR CYCLIP PRE TRAINED THAT I HAVE DOWNLOADED LOCALLY
model_clip_final, _, preprocess = open_clip.create_model_and_transforms(
    model_name="RN50",
    pretrained=None,  # Don't load default weights
    precision='fp32', # or 'amp' for mixed precision
    device=device
)

In [62]:
# Load the checkpoint - CLIP - Final
ckpt = torch.load("/kaggle/input/clip-final/clip_best.pt", map_location=device)
state_dict = ckpt["state_dict"]

# Remove 'module.' prefix from keys
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# Load into your model
model_clip_final.load_state_dict(new_state_dict)
model_clip_final.to(device)
model_clip_final.eval()

<ipython-input-62-74080bc77d25>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/kaggle/input/clip-final/clip_best.pt", map_location=device)


CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (

In [63]:
torch.save(new_state_dict, '/kaggle/working/clip_final.pt')


In [64]:
#FOR CYCLIP PRE TRAINED THAT I HAVE DOWNLOADED LOCALLY
model_Cyclip_final, _, preprocess = open_clip.create_model_and_transforms(
    model_name="RN50",
    pretrained=None,  # Don't load default weights
    precision='fp32', # or 'amp' for mixed precision
    device=device
)

In [65]:
# Load the checkpoint - CY CLIP - Final
ckpt = torch.load("/kaggle/input/cyclip-final/cyclip_best.pt", map_location=device)
state_dict = ckpt["state_dict"]

# Remove 'module.' prefix from keys
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# Load into your model
model_Cyclip_final.load_state_dict(new_state_dict)
model_Cyclip_final.to(device)
model_Cyclip_final.eval()

<ipython-input-65-681b61487b99>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/kaggle/input/cyclip-final/cyclip_best.pt", map_location=device)


CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (

In [66]:
torch.save(new_state_dict, '/kaggle/working/Cyclip_final.pt')


In [67]:
# open_clip.list_pretrained()

In [68]:
#                Models loaded are model_Cyclip_final and model_clip_final
#                Will further fine them below
                    # Will fine tune
                    #     --> as usual
                    #     --> with Dino regularizor

In [69]:
#          TO FINE TUNE vol1
# !python /kaggle/working/open-clip/src/open_clip_train/main.py \
#     --train-data /kaggle/working/train_data_karpathy.csv \
#     --name 'dino_fine_tuned_Clip_10_epochs' \
#     --dataset-type csv \
#     --csv-img-key image \
#     --csv-caption-key caption \
#     --csv-separator "," \
#     --model RN50  \
#     --pretrained /kaggle/working/clip_final.pt \
#     --batch-size 48 \
#     --lr 5e-6 \
#     --warmup 1000 \
#     --epochs 10 \
#     --lr-scheduler cosine \
#     --precision amp \
#     --workers 4 \
#     --logs "logs" \
#     --logs "checkpoints" \
#     --save-frequency 1 \
#     --seed 42 \
#     --lambda_dino 0.25 \
#     --alpha 0  \
#     --use_dino_reg 

In [70]:
# import torch
# import open_clip

# # Path to your trained checkpoint
# checkpoint_path = "checkpoints/dino_fine_tuned_Clip_10_epochs/checkpoints/epoch_10.pt"

# # 1. Create the model architecture from scratch (without loading pretrained weights)
# model_dino_fine_tuned_Clip_10_epochs, preprocess_ft, preprocess_val = open_clip.create_model_and_transforms(
#     "RN50", pretrained=None
# )

# # 2. Load the trained checkpoint manually
# checkpoint = torch.load(checkpoint_path, map_location="cpu",weights_only=True)

# # If the state_dict is nested under a key (like "state_dict"), adjust accordingly
# if "state_dict" in checkpoint:
#     checkpoint = checkpoint["state_dict"]

# # Load checkpoint weights into model
# model_dino_fine_tuned_Clip_10_epochs.load_state_dict(checkpoint, strict=False)  # Use strict=False to avoid missing keys error

# # 3. Move model to GPU (if available)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model_dino_fine_tuned_Clip_10_epochs = model_dino_fine_tuned_Clip_10_epochs.to(device)
# model_dino_fine_tuned_Clip_10_epochs.eval()
# print("Model loaded successfully!")

In [71]:
# TO LOAD EPOCH 6 and EPOCH 10
#LOAD EPOCH 10 at model_dino_fine_tuned_Clip_10_epochs

model_dino_fine_tuned_Clip_10_epochs, _, preprocess = open_clip.create_model_and_transforms(
    model_name="RN50",
    pretrained=None,  # Don't load default weights
    precision='fp32', # or 'amp' for mixed precision
    device=device
)

# Load from the checkpoint
ckpt = torch.load("/kaggle/input/10-epochs/model_dino_fine_tuned_Clip_10_epochs.pt", map_location=device)
state_dict = ckpt["state_dict"]

# Remove 'module.' prefix from keys
new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

# Load into your model
model_dino_fine_tuned_Clip_10_epochs.load_state_dict(new_state_dict)
model_dino_fine_tuned_Clip_10_epochs.to(device)
model_dino_fine_tuned_Clip_10_epochs.eval()

<ipython-input-71-7c329cfe231e>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/kaggle/input/10-epochs/model_dino_fine_tuned_Clip_10_epochs.pt", map_lo

CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (

In [80]:
#          TO FINE TUNE vol2 (4 epochs remaning)
!python /kaggle/working/open-clip/src/open_clip_train/main.py \
    --train-data /kaggle/working/train_data_karpathy.csv \
    --name 'og_fine_tuned_Clip_10_epochs' \
    --dataset-type csv \
    --csv-img-key image \
    --csv-caption-key caption \
    --csv-separator "," \
    --model RN50  \
    --pretrained /kaggle/input/10-epochs/model_og_fine_tuned_Clip_6_epochs.pt \
    --batch-size 48 \
    --lr 5e-6 \
    --warmup 1000 \
    --epochs 4 \
    --lr-scheduler cosine \
    --precision amp \
    --workers 4 \
    --logs "logs" \
    --logs "checkpoints" \
    --save-frequency 1 \
    --seed 42 \

    #--lambda_dino 0.25 \
    #--use_soft_labels \
    #--soft_temprature 0.05 \
    #--alpha 0  \
    #use_dino_reg

2025-04-02 10:37:14.297647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-02 10:37:14.317328: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-02 10:37:14.323416: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using device: cuda
['/kaggle/working/open-clip/src/open_clip_train/main.py', '--train-data', '/kaggle/working/train_data_karpathy.csv', '--name', 'og_fine_tuned_Clip_10_epochs', '--dataset-type', 'csv', '--csv-img-key', 'image', '--csv-caption-key', 'caption', '--csv-separator', ',', '--model', 'RN50', '--pretrained', '/kaggle/input/10-epochs/model_og_fine_tuned_

In [81]:
import torch
import open_clip

# Path to your trained checkpoint
checkpoint_path = "checkpoints/og_fine_tuned_Clip_10_epochs/checkpoints/epoch_4.pt"

# 1. Create the model architecture from scratch (without loading pretrained weights)
model_og_fine_tuned_Clip_10_epochs, preprocess_ft, preprocess_val = open_clip.create_model_and_transforms(
    "RN50", pretrained=None
)

# 2. Load the trained checkpoint manually
checkpoint = torch.load(checkpoint_path, map_location="cpu",weights_only=True)

# If the state_dict is nested under a key (like "state_dict"), adjust accordingly
if "state_dict" in checkpoint:
    checkpoint = checkpoint["state_dict"]

# Load checkpoint weights into model
model_og_fine_tuned_Clip_10_epochs.load_state_dict(checkpoint, strict=False)  # Use strict=False to avoid missing keys error

# 3. Move model to GPU (if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_og_fine_tuned_Clip_10_epochs = model_og_fine_tuned_Clip_10_epochs.to(device)
model_og_fine_tuned_Clip_10_epochs.eval()
print("Model loaded successfully!")

Model loaded successfully!


In [82]:
#!rm -rf checkpoints/og_fine_tuned_Clip_10_epochs
#!ls checkpoints/

In [83]:
# import shutil

# shutil.make_archive("/kaggle/working/cyclip_checkpoints", 'zip', "/kaggle/working/checkpoints")


In [84]:
#      EVALUATING BELOW

In [85]:
test_data = [item for item in karpathy_data["images"] if item["split"] == "test"]
all_captions = []
all_images = []
for item in test_data:
    for sentence in item["sentences"]:
        all_captions.append(sentence["raw"])
        all_images.append(item["filename"])

In [86]:
#FOR FINE TUNED CLIP

In [87]:
# Compute embeddings

image_root = "/kaggle/input/flickr30k/Images"
text_embeds, image_embeds = get_all_embeddings(
    model_dino_fine_tuned_Clip_10_epochs, all_captions, all_images, root=image_root,
    preprocess=preprocess, tokenizer=tokenizer, device=device , batch_size=48
)

Encoding batches: 100%|██████████| 105/105 [01:15<00:00,  1.40it/s]


In [88]:
# Evaluate
results = itm_eval(text_embeds, image_embeds)
print("📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.2f}")

📊 Evaluation Results:
txt_r1: 91.66
txt_r5: 96.74
txt_r10: 98.08
txt_r_mean: 95.49
img_r1: 43.06
img_r5: 73.62
img_r10: 82.62
img_r_mean: 66.43
r_mean: 80.96


In [89]:
image_root = "/kaggle/input/flickr30k/Images"
text_embeds, image_embeds = get_all_embeddings(
    model_og_fine_tuned_Clip_10_epochs, all_captions, all_images, root=image_root,
    preprocess=preprocess, tokenizer=tokenizer, device=device , batch_size=48
)

Encoding batches: 100%|██████████| 105/105 [01:04<00:00,  1.63it/s]


In [90]:
# Evaluate
results = itm_eval(text_embeds, image_embeds)
print("📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.2f}")

📊 Evaluation Results:
txt_r1: 91.38
txt_r5: 96.54
txt_r10: 98.06
txt_r_mean: 95.33
img_r1: 42.30
img_r5: 73.08
img_r10: 82.38
img_r_mean: 65.92
r_mean: 80.62


In [102]:
# CONSISTENCY METRICS BELOW

In [103]:
test_records = []
for item in test_data:
    img_filename = f"/kaggle/input/flickr30k/Images/{item['filename']}"
    for sentence in item["sentences"]:
        caption = sentence["raw"]
        test_records.append({"image": img_filename, "caption": caption})

In [104]:
from torch.utils.data import Dataset

class CyCLIPDataset(Dataset):
    def __init__(self, records, image_transform):
        self.records = records
        self.image_transform = image_transform

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        record = self.records[idx]
        image = self.image_transform(Image.open(record["image"]).convert("RGB"))
        text = tokenizer(record["caption"]).squeeze(0)  # remove batch dim
        return image, text

In [105]:
from torch.utils.data import DataLoader

test_dataset = CyCLIPDataset(test_records, preprocess)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [106]:
# compute_consistency_score calculate mean of similarities

# CLIP final with og Fine tuning

score = compute_consistency_score(model_dino_fine_tuned_Clip_10_epochs, test_loader, device)

print(f"Consistency Score: {score:.4f}")

Consistency Score: 0.4832


In [107]:
# CLIP final with og Fine tuning

score = compute_consistency_score(model_og_fine_tuned_Clip_10_epochs, test_loader, device)

print(f"Consistency Score: {score:.4f}")

Consistency Score: 0.4725
